In [26]:
import requests
import pandas as pd

class ScriptData:
    def __init__(self, api_key):
        self.api_key = api_key
        self.data = {}
    def fetch_intraday_data(self, script):
        response = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=1min&apikey={self.api_key}")
        data = response.json()
        self.data[script] = data['Time Series (1min)']
    
    def convert_intraday_data(self, script):
        intraday_data = self.data[script]
        df = pd.DataFrame(columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        for timestamp, info in intraday_data.items():
            df = df.append({
                'timestamp': pd.Timestamp(timestamp),
                'open': float(info['1. open']),
                'high': float(info['2. high']),
                'low': float(info['3. low']),
                'close': float(info['4. close']),
                'volume': int(info['5. volume'])
            }, ignore_index=True)
        self.data[script] = df
    
    def __getitem__(self, script):
        return self.data[script]
    
    def __setitem__(self, script, data):
        self.data[script] = data
    
    def __contains__(self, script):
        return script in self.data    

In [27]:
api_key = "LQGZHG47Y7RXROG4"
sd = ScriptData(api_key)

In [28]:
script = 'GOOGL'
sd.fetch_intraday_data(script)
sd.convert_intraday_data(script)
sd[script]


/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_39021/3601046383.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_39021/3601046383.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_39021/3601046383.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_39021/3601046383.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_3

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,94.18,94.18,94.11,94.12,3222
1,2023-02-14 19:59:00,94.17,94.18,94.17,94.18,3081
2,2023-02-14 19:58:00,94.16,94.18,94.16,94.18,1594
3,2023-02-14 19:56:00,94.18,94.18,94.15,94.15,1277
4,2023-02-14 19:55:00,94.22,94.22,94.18,94.18,614
...,...,...,...,...,...,...
95,2023-02-14 17:40:00,94.39,94.39,94.35,94.35,884
96,2023-02-14 17:39:00,94.39,94.4,94.39,94.4,502
97,2023-02-14 17:38:00,94.37,94.4,94.37,94.4,408
98,2023-02-14 17:36:00,94.36,94.36,94.36,94.36,2632


In [4]:
'GOOGL' in sd

True

In [5]:
'NVDA' in sd

False

In [6]:
df = sd[script]

In [7]:
def indicator1(df, timeperiod):
    result = pd.DataFrame(columns=['timestamp', 'indicator'])
    result['timestamp'] = df['timestamp']
    result['indicator'] = df['close'].rolling(timeperiod).mean()
    return result


In [8]:
# Example usage of the indicator1 function
timeperiod = 5
df_indicator1 = indicator1(df, timeperiod)
print(df_indicator1)

              timestamp  indicator
0   2023-02-14 20:00:00        NaN
1   2023-02-14 19:59:00        NaN
2   2023-02-14 19:58:00        NaN
3   2023-02-14 19:56:00        NaN
4   2023-02-14 19:55:00     94.162
..                  ...        ...
95  2023-02-14 17:40:00     94.374
96  2023-02-14 17:39:00     94.374
97  2023-02-14 17:38:00     94.378
98  2023-02-14 17:36:00     94.380
99  2023-02-14 17:35:00     94.374

[100 rows x 2 columns]


In [9]:
import pandas as pd

class Strategy:
    def __init__(self, script):
        self.script = script
        self.sd = ScriptData(api_key)
        self.df = None
        self.indicator_data = None
        self.signals = None
        
    def get_script_data(self):
        self.sd.fetch_intraday_data(self.script)
        self.sd.convert_intraday_data(self.script)
        self.df = self.sd[self.script]
    
    def get_indicator_data(self, timeperiod):
        self.indicator_data = indicator1(self.df, timeperiod)
    
    def get_signals(self):
        if self.df is None or self.indicator_data is None:
            print("Error: Script data or indicator data not available. Please call get_script_data and get_indicator_data first.")
            return
        
        # Create a copy of the indicator data and add a new column for the close data
        merged_data = self.indicator_data.copy()
        merged_data['close_data'] = self.df['close']
        
        # Create a new column for the signal based on the crossover of the indicator and close data
        merged_data['signal'] = 'NO_SIGNAL'
        merged_data.loc[merged_data['indicator'] > merged_data['close_data'], 'signal'] = 'BUY'
        merged_data.loc[merged_data['indicator'] < merged_data['close_data'], 'signal'] = 'SELL'
        
        self.signals = merged_data[['timestamp', 'signal']]
        
    def print_signals(self):
        if self.signals is None:
            print("Error: Signals not available. Please call get_signals first.")
            return
        
        # Only show rows where signal is 'BUY' or 'SELL'
        filtered_signals = self.signals[(self.signals['signal'] == 'BUY') | (self.signals['signal'] == 'SELL')]
        print(filtered_signals)


In [30]:
strategy = Strategy('NVDA')
strategy.get_script_data()
strategy.get_indicator_data(5)
strategy.get_signals()
strategy.print_signals()

/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_39021/3601046383.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_39021/3601046383.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_39021/3601046383.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_39021/3601046383.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_3

              timestamp signal
4   2023-02-14 19:56:00   SELL
5   2023-02-14 19:55:00    BUY
6   2023-02-14 19:54:00    BUY
8   2023-02-14 19:52:00   SELL
9   2023-02-14 19:51:00   SELL
..                  ...    ...
95  2023-02-14 18:12:00   SELL
96  2023-02-14 18:11:00   SELL
97  2023-02-14 18:10:00   SELL
98  2023-02-14 18:09:00    BUY
99  2023-02-14 18:07:00   SELL

[95 rows x 2 columns]


/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_39021/3601046383.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_39021/3601046383.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_39021/3601046383.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_39021/3601046383.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/xn/bctnwmy11mz7rfyx_fglsmpr0000gn/T/ipykernel_3